### 不使用主动空间近似优化的前提下
#### 目的:选取30%的UCCD 和70%的UCCGD 构成算符池

In [1]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer,FreezeCoreTransformer
import numpy as np
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Estimator
from qiskit.algorithms.minimum_eigensolvers import AdaptVQE
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

from qiskit.circuit import QuantumCircuit
from copy import  deepcopy
from qiskit.circuit import Parameter,ParameterVector
import sys
sys.path.append('..')
from AdaptVQE_.My_adapt import MyG_AdaotVQE
# Define Molecule
molecule = MoleculeInfo(
    ["Li", "H"], [(0.0, 0.0, 0.0), (1.7, 0.0, 0.0)],
    multiplicity=1,  # = 2*spin + 1
    charge=0,
)
driver = PySCFDriver().from_molecule(molecule)

problem = driver.run()
converter = QubitConverter(JordanWignerMapper(), two_qubit_reduction=True)
transformer = FreezeCoreTransformer(
    freeze_core=True,  # how many electrons we have in our active space
    # how many orbitals we have in our active space
    remove_orbitals=[-3, -2]
)
#problem = transformer.transform(problem=problem)

In [ ]:
from qiskit_nature.second_q.circuit.library import UCC
from qiskit.circuit.library import EvolvedOperatorAnsatz
uccsd = UCC(num_spatial_orbitals=problem.num_spatial_orbitals,num_particles=problem.num_particles,excitations='sd',\
    generalized=False)
uccgsd = UCC(num_spatial_orbitals=problem.num_spatial_orbitals,num_particles=problem.num_particles,excitations='sd',\
    generalized=True)
uccgd = UCC(num_spatial_orbitals=problem.num_spatial_orbitals,num_particles=problem.num_particles,excitations='d',\
    generalized=True)

uccsd_secondop = [converter.convert(i) for i in uccsd.excitation_ops()]
uccgsd_secondop = [converter.convert(i) for i in uccgsd.excitation_ops()]
uccgd_secondop = [converter.convert(i) for i in uccgd.excitation_ops()]
##
uccsd_secondop_pool = [EvolvedOperatorAnsatz(operators=i, insert_barriers=True, name='SD_'+str(index), parameter_prefix='SD_'+str(index)).to_instruction()\
                                                for index, i in enumerate(uccsd_secondop)]
uccgsd_secondop_pool = [EvolvedOperatorAnsatz(operators=i, insert_barriers=True, name='GSD_'+str(index), parameter_prefix='GSD_'+str(index)).to_instruction()\
                                                for index, i in enumerate(uccgsd_secondop)]
uccgd_secondop_pool = [EvolvedOperatorAnsatz(operators=i, insert_barriers=True, name='GD_'+str(index), parameter_prefix='GD_'+str(index)).to_instruction()\
                                                for index, i in enumerate(uccgd_secondop)]



In [ ]:
# a = MyAdaptVQE(ES_problem=problem)
uccsd_ansatz = MyAdapt_pool(ES_problem=problem,custom_operation_pool=None,excitation_index='sd') #92
uccgsd_ansatz = MyAdapt_pool(ES_problem=problem,custom_operation_pool=None,excitation_index='gsd') #285
uccgd_ansatz = MyAdapt_pool(ES_problem=problem,custom_operation_pool=None,excitation_index='gd')  #255
uccd_ansatz = MyAdapt_pool(ES_problem=problem,custom_operation_pool=None,excitation_index='d')  #76
print(f'UCCSD:{len(uccsd_ansatz.excitation_pool)},UCCD:{len(uccd_ansatz.excitation_pool)},UCCGD:{len(uccgd_ansatz.excitation_pool)}')

#### 筛选前X名的UCCD算符块
使用hf态作为初始态 计算和接下来的所有算符的下降梯度 选取更大的入选  
STEP 1:找到UCCD UCCGD UCCSD 中梯度比较大的算符 (标准？)

In [2]:
a = MyG_AdaotVQE(ES_problem=problem)

UCCD算符池筛选完毕!共入选26个,阈值是1e-15,具体index:
[69, 66, 45, 51, 60, 3, 42, 14, 24, 33, 18, 47, 28, 56, 30, 6, 34, 41, 62, 10, 2, 72, 7, 38, 65, 37],
            入选率:34.21052631578947%
UCCGD算符池筛选完毕!共入选24个,阈值是1e-15,具体index:
[142, 139, 96, 126, 110, 25, 93, 58, 42, 74, 62, 106, 46, 122, 71, 28, 92, 75, 135, 32, 29, 89, 138, 78],
            入选率:9.411764705882353%
正在检验是否有选中的UCCD&UCCGD算符是否有重叠..
算符确实有重叠!


In [3]:
a.final_operator_pool

[Instruction(name='D_2', num_qubits=12, num_clbits=0, params=[ParameterVectorElement(D_2[0])]),
 Instruction(name='D_72', num_qubits=12, num_clbits=0, params=[ParameterVectorElement(D_72[0])])]

In [ ]:
###UCCGD的所有算符 第一步的下降梯度 
n = uccgd_ansatz.init_state_hf.num_qubits
pre_circuit=[]
for i in uccgd_ansatz.excitation_pool_instruction:
    qc = QuantumCircuit(n)
    qc.append(uccgd_ansatz.init_state_hf,range(n))
    qc.append(i,range(n))
    pre_circuit.append(qc)

estimator = Estimator()
job = estimator.run(circuits=pre_circuit,observables=uccgd_ansatz.commutors,parameter_values=[[0.0]]*len(pre_circuit))
result = job.result()


In [ ]:
abs_value = np.abs(result.values)
np_sort = np.argsort(np.abs(result.values))
np_sort = np_sort[::-1]
#print(np.abs(result.values))
print(f'{np_sort}')
for index,i in enumerate(np_sort[0:20]):
    print(f'第{index}项的值为{abs_value[i]}')

In [ ]:
result.values[189] #-0.25020478036677607

In [ ]:
len(d4)
len(difference4)

In [ ]:
uccd_ansatz.excitation_pool[0].primitive.paulis

In [ ]:
2e-3

In [ ]:
n = uccgd_ansatz.init_state_hf.num_qubits
pre_circuit=[]
for i in uccgd_ansatz.excitation_pool_instruction:
    qc = QuantumCircuit(n)
    qc.append(uccd_ansatz.init_state_hf,range(n))
    qc.append(i,range(n))
    pre_circuit.append(qc)

estimator = Estimator()
job = estimator.run(circuits=pre_circuit,observables=uccd_ansatz.commutors,parameter_values=[[0.0]]*len(pre_circuit))
result = job.result()


In [ ]:
rank_index = np.argsort(np.abs(result.values)) #从小到大排列的
len(rank_index) #76
rank_index

In [ ]:
len(uccsd_ansatz.excitation_pool) #92
len(uccgsd_ansatz.excitation_pool) #285
len(uccgd_ansatz.excitation_pool) #255
len(uccd_ansatz.excitation_pool) #76

In [ ]:
print(rank_index1 == rank_index2)

In [ ]:
rank_index1 = np.argsort(result.values)

In [ ]:
rank = np.argsort(np.abs(val))
len(rank[0])

In [ ]:
rank = np.argsort(val[0])

In [ ]:
uccsd_secondop

In [ ]:
uccsd.excitation_ops()

In [4]:
1 in [1,2,3] 

True